# CS 336 Lecture 3.2 Hyperparameter & Training Techniques for Large Language Models

## Hyperparameter for FFN Layer

For the feed forward layer for the transformer block:

$$\text{FFN}(x) = \max(0, (xW_1 + b_1)W_2 + b_2)$$

The dimension of this block is a kind of hyperparameter settings:

$$d_{\text{ff}} = 4 \times d_{\text{model}}$$

### Variants: GLU

Remember that GLU variants scale down by 2/3rd. This means most GLU variants have $d_{\text{ff}} = 8/3 \times d_{\text{model}}$


### Variants: T5

$$d_{\text{ff}} =  65536$$

$$d_{\text{model}} = 1024$$

![Hyperparameter for FFN Layer](../../assets/imgs/ffn.png)

## Ratio for Transformer Blocks

在 Transformer 模型中，一般使用两个参数量来描述模型本身：

- 模型的**深度**：即 $\text{Layer}$ 的层数，在解码器中模型会堆叠多少层的结构。
- 模型的**宽度**：即在多头注意力层中 $d_{\text{model}}$ 的层数和 FFN 层中的隐藏层的维度。当然也包括多头注意力中头的数量。

Several Model Config Settings:

| | Num heads | Head dim | Model dim | Ratio |
| :--- | :--- | :--- | :--- | :--- |
| **GPT3** | 96 | 128 | 12288 | 1 |
| **T5** | 128 | 128 | 1024 | 16 |
| **T5 v1.1** | 64 | 64 | 4096 | 1 |
| **LaMDA** | 128 | 128 | 8192 | 2 |
| **PaLM** | 48 | 258 | 18432 | 1.48 |
| **LLaMA2** | 64 | 128 | 8192 | 1 |

在这些参数设定中，$\text{ratio} = \frac{d_{\text{model}}}{n_{\text{layer}}}$ 是非常重要的一个模型参数。

### Aspects for parallelism

-  因为 Transformer 的层数的堆叠依赖线性的输入顺序，因此难以并行。只能做数据并行，而不能把不同的层放在不同的设备上同时做训练。

- 而提升宽度对并行化来说即为容易：当维度非常大时，你可以将一个大的矩阵乘法（例如，Weight × Input）操作分解成多个小的矩阵乘法，然后将这些小的操作分配到不同的设备上并行计算。这种技术被称为张量并行（Tensor Parallelism）或流水线并行（Pipeline Parallelism），它可以有效地利用多台机器或多个 GPU。






## Regularization

For the pre-training process, do we need dropout and other regularization method?

- DropOut

- WeightDecay: L2 Regularization penalty.


### Why Weight Decay LLMs?

Relevant Papers: https://arxiv.org/abs/2310.04415

For the amount of training data during pre-training much more than the total amount of parameters, thus there is no need for large language models to prevent overfitting.

However, the adjust of learning rate during training is quite important, and weight decay (such as cos LR weight decay) can help adjust learning rate and optimize losses at the end of the pre-training process.


#### Cos LR Decay

**余弦退火学习率调度（Cosine Annealing Learning Rate Schedule）**，这是一种非常流行的学习率衰减策略。它通过一个余弦函数来调整学习率，而不是像传统的步长衰减（Step Decay）或指数衰减（Exponential Decay）那样进行线性或阶梯式的衰减。在深度学习的训练中，我们通常希望在训练初期使用较大的学习率，以快速接近损失函数的局部或全局最优解。随着训练的进行，为了更精确地收敛到最优解，我们需要逐渐减小学习率，以避免在最优解附近来回震荡。余弦退火策略正是利用了这一点，它将学习率的变化过程模拟成余弦函数曲线的一部分。


余弦退火学习率的计算公式通常如下：

$$\eta_t = \eta_{min} + \frac{1}{2}(\eta_{max} - \eta_{min})(1 + \cos(\frac{T_{cur}}{T_{max}}\pi))$$

其中：
* $\eta_t$：在当前训练步（或 epoch）$t$ 的学习率。
* $\eta_{min}$：学习率的最小值。
* $\eta_{max}$：学习率的最大值（通常是初始学习率）。
* $T_{cur}$：当前训练步数（或 epoch）。
* $T_{max}$：整个衰减周期的总步数（或总 epoch 数）。

余弦函数的特点是：在开始时下降缓慢，在中间阶段下降速度最快，在结束时再次下降缓慢。

当我们将这个函数应用于学习率时，会得到一个非常平滑的衰减曲线：
* **训练初期**：学习率下降缓慢，允许模型进行大步的参数更新，快速探索解空间。
* **训练中期**：学习率下降加速，使得模型能够更稳定地收敛。
* **训练末期**：学习率再次下降缓慢，允许模型在最优解附近进行精细调整，从而找到更精确的解。

这种平滑的衰减模式被认为比简单的线性或阶梯式衰减更有助于模型的稳定训练和最终收敛。

余弦退火策略最早在论文 **"SGDR: Stochastic Gradient Descent with Warm Restarts"** 中被提出。这篇论文不仅提出了余弦退火，还引入了**周期性重启（Periodic Restarts）**的概念。

* **SGDR** 的核心思想是，每隔一段时间，将学习率突然重置回初始的最大值，然后再次按照余弦函数衰减。
* 这就像是训练过程中给模型一个“冲击”，帮助它从局部最优解中“跳出”，从而有机会找到更好的全局最优解。

因此，当提到“Cos LR decay”时，它可能指：
1.  **单纯的余弦退火**：学习率从最大值平滑衰减到最小值，直到训练结束。
2.  **带重启的余弦退火（SGDR）**：学习率周期性地进行余弦衰减和重启。

## Stable Training

For the pre-training process, training stability is quite important.

### Be aware of softmax

While the softmax is combined with cross-entropy loss, it will cause unstability!

- **Overflow** for exponential: when dealing with large number input $x$, the exponential result for $e^x$ may be extremely large. (Larger than the upper bound for the float number)

- `inf` problems: divide by zero.

### z-loss

回顾 Softmax 的基础公式：

$$P_i = \frac{e^{z_i}}{\sum^{K}_{j=1} e^{z_j}}$$

$$\log P_i = z_i - \log (\sum^{K}_{j=1} e^{z_j})$$

对于输入样本 $x$，定义维度 $i$ 的值 $z_i$ 为 $U_{i}(x)$，最终 Softmax 输出的分布向量为 $P(x)$，则公式可以被修正为：

$$P(x)_r = U_r(x) - \log (\sum^{|V|}_{r' = 1} U_{r'}(x))$$

但是当Softmax 和二进制交叉熵损失放在一起之后，就会出现上述的训练不稳定的问题（归根到底还是基于梯度的优化器对数值范围非常敏感），因此，对于优化过程，需要**调整损失函数**来提升稳定性。

对于原始的二进制交叉熵损失函数：

$$L_{\text{CE}} = \sum_{i} \log P(x_i)$$

> 这里没有加符号，因为是最小化损失。

Z-loss 的核心是惩罚 Softmax 归一化常数 Z 的对数，即 $\log^2(Z) = \log^2(\sum^{K}_{j=1} e^{z_j})$。通过将这个惩罚项加入到总损失函数中，模型在训练过程中会被激励去限制 logits 的整体幅度，防止它们变得过大。这样就可以一定程度避免因为指数项过大导致的溢出问题。

因此，在 Z-loss 中，加入了这一项惩罚项：

$$L_{\text{ZLoss}} = \alpha \sum_i\log^2(Z(x_i))$$

$$L' = \sum_{i} \log P(x_i) - \alpha \sum_i\log^2(Z(x_i))$$

### QK-Norm

Normalization 本身就是提升训练稳定性的一种有效的方式，因此一种很常见的技巧是在 Attention 的核心模块，对Q,K矩阵先做一次 RMS Norm，在送入批量矩阵乘法之后再做 softmax。这个步骤被嵌入了多头注意力的模块内部。

![QK Norm](../../assets/imgs/qknorm.png)

### Logit SoftCapping

同样是基于 Softmax 的大数溢出问题的优化，使用 $\tanh$ 函数。

具体而言，对于 Softmax 操作之前做如下的优化：

$$x \longleftarrow \text{soft cap} \times \tanh(\frac{x}{\text{soft cap}})$$

对于较小的输入，因为 $\tanh$ 在 $x = 0$ 处的点近似线性，因此不受影响，运算几乎等价，但是在远处的点受限于函数本身的值域限制，就保证了在 Softmax 层输入的时候不会有太大的数。同时引入超参数 $\text{soft cap}$ 可以调节缩放的scaling。

## Attention Heads Optimization

### Perplexity 大语言模型的困惑度

大模型的**困惑度（Perplexity）**是一种衡量语言模型**好坏**的重要指标，它量化了模型对一个给定的文本序列预测的**不确定性**或**困惑程度**。**困惑度越低，说明模型的预测越准确，泛化能力越强。**


困惑度是**交叉熵（Cross-Entropy）**的指数形式，**交叉熵（Cross-Entropy）**：用于衡量两个概率分布之间的差异。在语言模型中，它衡量的是模型**预测的词汇分布**与**真实词汇分布**（通常是独热编码）之间的差异。**交叉熵越小**，说明模型预测的概率分布与真实分布越接近，模型性能越好。

给定一个包含 $N$ 个词的测试集文本序列 $W = (w_1, w_2, \dots, w_N)$，该序列在语言模型上的困惑度 **$PP(W)$** 的定义为：

$$PP(W) = P(w_1, w_2, \dots, w_N)^{-\frac{1}{N}}$$

其中，$P(w_1, w_2, \dots, w_N)$ 是模型对整个序列的联合概率。

这个联合概率可以根据**链式法则**分解为一系列条件概率的乘积：
$$P(W) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1, w_2) \cdots P(w_N|w_1, \dots, w_{N-1})$$

将这个分解后的公式代入，我们得到：

$$PP(W) = \left( \prod_{i=1}^{N} P(w_i|w_1, \dots, w_{i-1}) \right)^{-\frac{1}{N}}$$

> 直观理解就是模型输出目标句子的概率乘积。乘上 $1/N$ 的指数是为了抵消句子长度对困惑度计算的影响。（对于 $N$ 较大的测试集，肯定最终的概率计算值会更小，但是开根 $1/N$ 后值会变大，使这个指标**和序列无关**），最后**取倒数**是为了让困惑度和对应指标成正比（越困惑，模型的数越大）

困惑度通常通过对数形式来计算，这在数值上更稳定。我们先计算平均交叉熵损失 $L(W)$：

$$L(W) = -\frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i|w_1, \dots, w_{i-1})$$

然后，困惑度就是这个平均交叉熵的 2 的指数：

$$PP(W) = 2^{L(W)}$$

因此，困惑度 = $2^{\text{平均交叉熵}}$。

* **直观理解**：困惑度可以被看作是语言模型在每个位置**平均**面临的**“有效”选择数量**。
    * 为什么？因为困惑度和 $N$ 无关，取 $N=1$，则可以得到联合概率的值为 $1/100$，平均来说就是有100种可能性。
    * 如果一个模型的困惑度是 100，这意味着该模型在预测下一个词时，平均来说有 100 个同样可能的词汇可供选择。
    * 困惑度越低，模型在每个位置的不确定性越小，其预测能力就越强。
    * 困惑度为 1 是最理想的情况，这代表模型每次都以 100% 的概率预测出正确的词，这在现实中是不可能达到的。

* **与模型性能的关系**：
    * **低困惑度**：意味着模型对测试集数据的分布有很好的理解，能够给出高概率的正确预测，因此它是一个**好**的模型。
    * **高困惑度**：意味着模型对测试集数据的预测能力较差，对下一个词的预测存在很高的不确定性，因此它是一个**差**的模型。

> 在实际应用中，由于大模型通常以 **token**（而不是词）为单位进行处理，所以困惑度计算也通常基于 token 序列。

### GQA & MQA

Let consider the compute cost (at inference time) while the input passing the attention layer.

![image](https://s1.imagehub.cc/images/2025/08/23/206bc6dfbd02b78ebadfa2335b4f54ad.png)

For the input $X$, we first do multi-head attention for transform into $Q,K,V$ matrix in different subspaces.

$$X \in \mathbb{R}^{b \times n \times d}, Q \in \mathbb{R}^{d \times d}$$

For this matrix multiplication, it time complexity is $O(b \times n \times d^2)$

For the total memory access:

- Q, K, V: $\in \mathbb{R}^{d \times d}$

- X and other matrix ($XQ$, $K^\top X^\top$): $\in \mathbb{R}^{b \times n \times d}$

- $XQK^\top X^\top$: $\in \mathbb{R}^{b \times n \times n}$ (And there are $n$ batches)

For the total memory access: $O(bnd + bhn^2 + d^2)$

Arithmetic intensity: $O((\frac{1}{bn} + \frac{1}{d} + \frac{hn}{d^2})^{-1})$

Hence, for text generation task, the generation is done step by step, we need to update the QKV matrices every time, it is quite memory-consuming! But for the generated text, some part of the matrix remain unchanged, thus we can use the cache to store it! **KVcache**

#### KV cache

The KV Cache optimizes autoregressive **inference** by exploiting the structure of the self-attention mechanism.

The core of the attention mechanism is:
$$Attention(Q, K, V) = \text{Softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$
Where $Q$, $K$, and $V$ are matrices derived from the input sequence $X$.

In autoregressive generation, the model predicts one token at a time. To predict the $i$-th token, the input is the entire sequence from token 1 to $i$. The model would need to compute new $K$ and $V$ matrices for the **entire sequence** at each step, even though the $K$ and $V$ vectors for the first $i-1$ tokens have already been calculated. This leads to **redundant computations**.

**The KV Cache Solution:**
The mathematical principle is based on a simple observation: at each step, a new token is added, which contributes only **one new row** to the $Q$, $K$, and $V$ matrices. The $K$ and $V$ vectors from previous tokens remain constant.

KV Cache leverages this by:
1.  **Caching Past Vectors**: After computing the $K$ and $V$ vectors for tokens $1$ to $i-1$, these vectors are stored in memory (the "cache").
2.  **Efficient Update**: To generate the $i$-th token, the model only needs to compute the **new** $K_i$ and $V_i$ vectors for the current token.
3.  **Concatenation**: The new vectors are **concatenated** with the cached past vectors to form the full $K$ and $V$ matrices for the current step:
    $$K_{full} = [K_{cache}, K_i]$$   $$V_{full} = [V_{cache}, V_i]$$
4.  **Optimized Computation**: The attention score is then calculated using only the **new query vector** $Q_i$ and the **full, concatenated** $K$ and $V$ matrices:
    $$Attention_{new} = \text{Softmax}\left(\frac{Q_i K_{full}^T}{\sqrt{d_k}}\right)V_{full}$$

In essence, KV Cache uses memory (space) to store previously computed $K$ and $V$ vectors, eliminating the need to re-compute them in each generation step. This significantly reduces the computational complexity from being a quadratic function of sequence length to a linear one, drastically speeding up inference.

#### MQA: Multi-Query Attention

对于多头注意力机制，在推理阶段使用内存的空间复杂度为：$O(hnd)$

为了减少内存占用率，MQA实现了从 Multi-Head 到 Multi-Query 的转化，即对于多个头不再使用不同的KV矩阵，而是使用相同的，只是对应的query有不同，这样可以把推理的空间复杂度降一个维度：$O(nd)$。

> 这样的设定机制是为了搭配 KVcache 的缓存技巧

但是这样会存在问题！在一开始的多头注意力中，原始的输入被线性映射到不同的子空间，按理来说其对应的KV矩阵也应该不同，而不能相同，因此这会带来预测上性能的下降。(相当于丢弃参数了)

因此：**GQA** 作为一种折中的方案出现了！

#### GQA

Just a trade of between the expressiveness of the model and the inference efficiency of the model.

![GQA](https://s1.imagehub.cc/images/2025/08/23/46fd8cd25f363ab395f399ecab03068a.png)

### Sparse Attention

Attending to the entire context can be expensive (quadratic). Build sparse / structured attention that trades off expressiveness vs runtime (GPT3)

对于传统的 Attention 模块，存在一个非常严重的时间瓶颈：当序列长度 $N$ 变得非常大的时候，注意力矩阵的时间复杂度计算达到了 $O(N^2)$ 级别。因此，同样为了达到 tradeoff（推理性能和推理时间的权衡），**稀疏注意力**在计算注意力分数时，只让每个 Token 关注序列中的一小部分其他 Token，而不是所有 Token。

稀疏注意力通过定义一个**注意力掩码**（attention mask）来实现，这个掩码会阻止一些不必要的注意力计算，只保留那些被认为最重要的连接。

#### Sliding Window Attention

有点类似于 CNN，构建滑动窗口，只捕捉序列滑动窗口的信息。但是这也让其难以捕捉长距离依赖关系。

##### LongFormer

https://arxiv.org/abs/2004.05150

![Sliding Window Attention](https://s1.imagehub.cc/images/2025/08/23/a8c023ddd7e1ce62f8c08235a179e732.png)

Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length. To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer. **Longformer's attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention**. Following prior work on long-sequence transformers, we evaluate Longformer on character-level language modeling and achieve state-of-the-art results on text8 and enwik8. In contrast to most prior work, we also pretrain Longformer and finetune it on a variety of downstream tasks. Our pretrained Longformer consistently outperforms RoBERTa on long document tasks and sets new state-of-the-art results on WikiHop and TriviaQA. We finally introduce the Longformer-Encoder-Decoder (LED), a Longformer variant for supporting long document generative sequence-to-sequence tasks, and demonstrate its effectiveness on the arXiv summarization dataset.

> Through ablations and controlled trials we show both attention types are essential – the local attention is primarily used to build contextual representations, while the global attention allows Longformer to build full sequence representations for prediction

> 不过这个仍然针对的是 seq2seq 的序列预测问题，因此还是采用了原始的编码器解码器架构（LED: Longformer-Encoder-Decoder）

#### Combination

因为 Transformer 块可以实现堆叠，因此每一个堆叠的 Transformer 块可以设计不同的结构，来同时捕捉长程和短程的序列信息。

![image](https://s1.imagehub.cc/images/2025/08/23/b222569bb35fda5ee86561d291db40cd.png)

#### Reformer

Large Transformer models routinely achieve state-of-the-art results on a number of tasks but training these models can be prohibitively costly, especially on long sequences. We introduce two techniques to improve the efficiency of Transformers. For one, we replace dot-product attention by one that uses **locality-sensitive hashing**, changing its complexity from O(L2) to O(L log L), where L is the length of the sequence. Furthermore, we use **reversible residual layers** instead of the standard residuals, which allows storing activations only once in the training process instead of N times, where N is the number of layers. The resulting model, the Reformer, performs on par with Transformer models while being much more **memory-efficient and much faster on long sequences**.

A lot more about formers for lightening attention and sparse attention! We will discuss it further in the future.